In [1]:
import os, sys, warnings, operator, itertools
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error as mse
def rmse(pred, true): return np.sqrt(mse(true, pred))

In [22]:
read_path = './dataset/origin/'
write_path = './dataset/submission/'
sample = pd.read_csv(read_path+'sample.csv')
train = pd.read_csv(read_path+'train.csv')
test = pd.read_csv(read_path+'test_mini.csv')

## 타겟과 로스

1. 진짜 타겟(12개의 합)을 타겟으로 삼는다 -> lgb로 망함 ㅋ
2. 12개를 각각 타겟으로 삼는다 -> lgb로 망함 ㅋ

---
1. 그냥 2014년도 자료를 제출한다 -> 316.439768872516
3. 자살자 수 변화율을 타겟으로 삼는다 -> 200.62501063108937
4. 자살자 수 변화율12 -> 196.00703758110848

---
5. 자살율 변화율을 타겟으로 삼는다
6. 자살율 변화율12
1. 자살자 수 변화량 (처음에 했던 거에서 조금 진보)
1. 자살자 수 변화량 12 
1. 자살율 변화량
1. 자살율 변화량 12

## 그냥 전년도와 같은 수치를 제출했을 때

In [ ]:
testCountries = test['country'].unique().tolist()
train_pure = train[train['country'].apply(lambda x: x in testCountries)]
train_pure = train_pure[train_pure['year']==2014]
train_pure = train_pure.groupby('country')['suicides_no'].agg('sum').reset_index()
train_pure.columns = ['country', 'y_pred']
train_pure = train_pure.sort_values(by='country', ascending=True)
train_pure.to_csv(write_path+'01_same_as_2014.csv', index=False)

## 전전년도와 전년도의 자살자 수 변화율을 전년도 타겟에 곱했을 때

In [ ]:
train_pure = train[train['country'].apply(lambda x: x in testCountries)]
train_2014 = train_pure[train_pure['year']==2014]
train_2013 = train_pure[train_pure['year']==2013]
ukraine = train_pure[(train_pure['country']=='Ukraine') & (train_pure['year']==2012)]
train_2013 = train_2013.append(ukraine)
cols = ['country', 'sex', 'age']
train_2014 = train_2014.sort_values(by=cols)
train_2013 = train_2013.sort_values(by=cols)
train_2013 = train_2013.reset_index(drop=True)
train_2014 = train_2014.reset_index(drop=True)

### 12개 각각 구하기

In [ ]:
def diff_btw_2013_2014(x):
    _2014 = x['suicides_no_2014']
    _2013 = x['suicides_no_2013']
    try : return (_2014-_2013)/_2013+1
    except ZeroDivisionError : return np.nan
    
def target(x):
    rate = x['diff_btw_2013_2014']
    if pd.isna(rate): return x['suicides_no_2014']
    else : return rate*x['suicides_no_2014']

In [77]:
cols = ['country', 'suicides_no', 'year']
train_2014 = train_2014[cols]
train_2013 = train_2013[cols]
train_2014.columns = cols[:-2]+[col+'_2014' for col in cols[1:]]
train_2013.columns = cols[:-2]+[col+'_2013' for col in cols[1:]]
train_20134 = pd.concat([train_2014, train_2013[[col+'_2013' for col in cols[1:]]]],axis=1)
train_20134['diff_btw_2013_2014'] = train_20134.apply(diff_btw_2013_2014, axis=1)
train_20134['y_pred'] = train_20134.apply(target, axis=1)
train_20134_agg = train_20134.groupby('country')['target'].agg('sum').reset_index()

train_20134_agg.to_csv(write_path+'02_diff_btw_2013_2014_cbc.csv', index=False)

### 합친채로 구하기

In [104]:
cols = ['country', 'suicides_no', 'year']
train_2014 = train_2014[cols]
train_2013 = train_2013[cols]

train_2014_agg = train_2014.groupby('country')['suicides_no'].agg('sum').reset_index()
train_2013_agg = train_2013.groupby('country')['suicides_no'].agg('sum').reset_index()
train_2014_agg.columns = ['country', 'suicides_no_2014']
train_2013_agg.columns = ['country', 'suicides_no_2013']

train_2014_agg['suicides_no_2013'] = train_2013_agg['suicides_no_2013']
train_2014_agg['diff_btw_2013_2014'] = train_2014_agg.apply(diff_btw_2013_2014, axis=1)
train_2014_agg['y_pred'] = train_2014_agg.apply(target, axis=1)

train_2014_agg[['country', 'y_pred']].to_csv(write_path+'02_diff_btw_2013_2014_total.csv', index=False)